# Gather

In [ ]:
import pandas as pd
import numpy as np
import tweepy as tp
import requests
import yaml
import os
import json
import time
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm as log_progress
import warnings

In [ ]:
tweets = pd.read_csv('data_files/twitter-archive-enhanced.csv')

In [ ]:
predictions_file_url = "https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv"
r = requests.get(predictions_file_url)
with open('data_files/image-predictions.tsv', 'wb') as f:
        f.write(r.content)
predictions = pd.read_csv('data_files/image-predictions.tsv','\t')

In [ ]:
parameters = yaml.load(open('parameters.yml'), Loader=yaml.BaseLoader)['parameters']
auth = tp.OAuthHandler(parameters['key'], parameters['key_secret'])
auth.set_access_token(parameters['token'], parameters['token_secret'])
api = tp.API(auth)

In [ ]:
skipped_tweets = []

with open('data_files/tweet_json.txt', 'w') as data_file, open ('data_files/tweet_json_failed.txt', 'w') as failed_data_file:
    for tweet_id in log_progress(tweets['tweet_id']):
        try:
            while True:
                try:
                    result = api.get_status(tweet_id)
                    break
                except tp.RateLimitError:
                    time.sleep(60)
            data_file.write(json.dumps(result._json))
            data_file.write('\n')
        except tp.TweepError as e:
            record = {'tweet_id':tweet_id, 'error':e.response.text}
            skipped_tweets.append(record)
            failed_data_file.write(json.dumps(record))
            failed_data_file.write('\n')

In [ ]:
api_data = []
with open('data_files/tweet_json.txt', 'r') as data_file:
    line = data_file.readline()
    while line:
        line = json.loads(line) 
        api_data.append({'tweet_id':line['id'], 'retweets': line['retweet_count'], 'likes': line['favorite_count']})
        line = data_file.readline()

        
reactions = pd.DataFrame(api_data)

<hr style="border:2px solid black"> </hr> 

# Asses

In [ ]:
warnings.filterwarnings('ignore')

In [ ]:
tweets.sample(3)

In [ ]:
tweets[tweets.name.str.len() < 3].name.value_counts()

In [ ]:
tweets.info()

In [ ]:
tweets.rating_denominator.value_counts()

In [ ]:
type(tweets.timestamp[0])

In [ ]:
tweets.tweet_id.duplicated().sum()

In [ ]:
tweets.name.value_counts()

In [ ]:
tweets[tweets.name.str.lower() == tweets.name].name.value_counts()

In [ ]:
for index, row in tweets[tweets.rating_denominator != 10].iterrows():
    print('{} , {}\n {} \n'.format(tweets.loc[index, ['rating_numerator']], tweets.loc[index, ['rating_denominator']], tweets.loc[index, ['text']].iloc[0]))

In [ ]:
predictions.sample(4)

In [ ]:
predictions.info()

In [ ]:
predictions.p1_dog.value_counts()

In [ ]:
predictions[predictions.p1_dog == False].p2_dog.value_counts()

In [ ]:
predictions[predictions.p1_dog == False][predictions.p2_dog == False].p3_dog.value_counts()

In [ ]:
predictions.describe()

In [ ]:
predictions.p1.value_counts()

In [ ]:
reactions

In [ ]:
reactions.info()

In [ ]:
reactions.describe()

In [ ]:
reactions.duplicated().sum()

In [ ]:
try:
    skipped_tweets
except NameError:
    print('Skipped tweets is not defined')

In [ ]:
try:
    skipped_tweets[:5]
except NameError:
    print('Skipped tweets is not defined')

In [ ]:
tweets[tweets.tweet_id == 888202515573088257].text.iloc[0]

In [ ]:
tweets[tweets.tweet_id == 775096608509886464].text.iloc[0]

In [ ]:
tweets[tweets.tweet_id == 775096608509886464]

In [ ]:
tweets[tweets.tweet_id == 771004394259247104].text.iloc[0]

#### Quality
##### `tweets` 
- timestamp is a string
- `name`, `pupper`, `doggo`, `puppo`, `floofer`,  column is set to 'None' when there is no value 
- `name` column has a lot of incorrect values like `a` (55 records), `by`, `very`, `an`, `the`... etc
- `text` include urls which misleads the actual length of the tweet
- Issues with the extracted denominator:
    - a. wrongly extracted denominator, while a correct value does exist
    - b. a denominator has a value of 0
    - c. an incorrect value of 24/7 rating is extracted
    - d. inconsistent values

##### `reactions`
- 25 tweets missing like and retweet count (**Can't fix**)
- some records are retweets

#### Tidiness
- rating is represented in two columns
- dog category is splitted in 4 columns
- multiple predictions are available


<hr style="border:2px solid black"> </hr>

# Clean

In [ ]:
#make copies to preserve the original data
tweets_clean = tweets.copy()
predictions_clean = predictions.copy()
reactions_clean = reactions.copy()

### _Quality_

1.`tweets` timestamp is a string
#### Define
- convert timestamp to datetime

In [ ]:
# Code 

tweets_clean.timestamp = pd.to_datetime(tweets_clean.timestamp)

# extract month and year, would be useful later
tweets_clean['month_year'] = tweets_clean.timestamp.apply(lambda x: str(x.month) + '/' + str(x.year))

In [ ]:
# Test

print(type(tweets_clean.timestamp[0]))
print(tweets_clean.month_year[:3])

2.`tweets` name,pupper,doggo,puppo and floofer columns is set to 'None' when there is no value 

#### Define
- replace the string 'None' with Nan

In [ ]:
# Code 

tweets_clean.replace('None', np.nan, inplace=True)

In [ ]:
# Test 

assert len(tweets_clean[tweets_clean.name == 'None']) == 0
assert len(tweets_clean[tweets_clean.pupper == 'None']) == 0
tweets_clean.info()

3.`tweets` 'name' column has incorrect values 

#### Define
- replace the string 'a', 'an', 'by', etc with Nan (A useful note is that all caught invalid strings are in lower case)

In [ ]:
# Code 

values_to_remove = tweets_clean[tweets_clean.name.str.lower() == tweets.name].name.value_counts()
tweets_clean.replace(values_to_remove.keys(), np.nan, inplace=True)

In [ ]:
# Test 

tweets_clean[tweets_clean.name.str.lower() == tweets.name].name.value_counts()

4.`tweets` 'text' include urls which misleads the actual length of the tweet

#### Define
- remove urls from text if found

In [ ]:
# Code 

tweets_clean.text = tweets_clean.text.str.split('http', n =1, expand = True)[0]
tweets_clean.text = tweets_clean.text.str.strip()

In [ ]:
# Test 

print('Before : {}, After: {}'.format(tweets.text.str.contains('http').sum(), tweets_clean.text.str.contains('http').sum()))
tweets_clean.sample().text.iloc[0]

5.`tweets` Issues with the extracted denominator:
- a. wrongly extracted denominator, while a correct value does exist
- b. a denominator has a value of 0

#### Define
- re-extract a valid rating from the text

In [ ]:
# Code

not_10_denom_records = tweets_clean[tweets_clean.rating_denominator != 10]
rating_parts = not_10_denom_records.text.str.extract('([0-9]+)/10').dropna()
rating_parts
for index, row in rating_parts.iterrows():
    tweets_clean.rating_numerator[index] = row[0]
    tweets_clean.rating_denominator[index] = 10

In [ ]:
#Test 


assert len(tweets_clean[tweets_clean.rating_denominator == 0]) == 0
assert len(tweets[tweets.rating_denominator == 10]) < len(tweets_clean[tweets_clean.rating_denominator == 10])

5.`tweets` Issues with the extracted denominator:
- c. an incorrect value of 24/7 rating is extracted

#### Define
- drop the rating for this tweet

In [ ]:
# Code 

index_to_clean = tweets_clean[tweets_clean.rating_numerator == 24][tweets_clean.rating_denominator == 7].index
for i in index_to_clean:
    tweets_clean.rating_numerator[i] = np.nan
    tweets_clean.rating_denominator[i] = np.nan

In [ ]:
# Test

assert len(tweets_clean[tweets_clean.rating_numerator == 24][tweets_clean.rating_denominator == 7]) == 0

5.`tweets` Issues with the extracted denominator:
- d. rating denominators are not consistent

#### Define
- convert denominators to 100 and adjust the numenator accordingly

In [ ]:
# Code 

tweets_clean.rating_numerator = tweets_clean.rating_numerator*100.0/ tweets_clean.rating_denominator
tweets_clean.rating_denominator = 100

In [ ]:
# Test 

sample = tweets_clean.sample(10)
for index, row in sample.iterrows():
    assert int(row.rating_numerator/row.rating_denominator) == int(tweets.rating_numerator[index]/tweets.rating_denominator[index])

### _Tidiness_

1.`breeds are presented in multiple columns`
#### Define
- create two new columns containing the first dog breed availble and its conf or NaN if all predictions are not breeds

In [ ]:
# Code 

predictions_clean['breed'] = predictions_clean[predictions_clean.p1_dog].p1
predictions_clean['conf'] = predictions_clean[predictions_clean.p1_dog].p1_conf

for index, row in predictions_clean.iterrows():
    if pd.isna(row.breed):
        if row.p2_dog:
            predictions_clean.iloc[index, 12] = row.p2
            predictions_clean.iloc[index, 13] = row.p2_conf
        elif row.p3_dog:
            predictions_clean.iloc[index, 12] = row.p3
            predictions_clean.iloc[index, 13] = row.p3_conf

In [ ]:
# Test 

test_sample = predictions_clean.sample(10)
for index, row in test_sample.iterrows():
    print('({}, {}, {}) => \n {}\n'.format(row.p1, row.p2, row.p3 ,row.breed))

2.`rating is represented in two columns, the denominator is redundant`
#### Define
- drop the denominator and rename the numenator to rating

In [ ]:
# Code 

tweets_clean.drop(columns=['rating_denominator'], inplace=True)
tweets_clean.rename(columns={'rating_numerator': 'rating'}, inplace=True)

In [ ]:
# Test 

tweets_clean.sample()

3.`dog category is splitted in 4 columns`

#### Define
- Create a new column that has the type of the dog
- Drop category columns

In [ ]:
# Code

temp = tweets_clean.copy()
temp['index'] = temp.index.copy()
temp = temp.melt(id_vars=['index'], value_vars=['doggo', 'floofer', 'pupper', 'puppo']).dropna()
temp = temp.sort_values('index')
temp.set_index(temp['index'],inplace=True)
tweets_clean.loc[temp.index,'type']= temp['value']
tweets_clean.drop(columns=['doggo', 'floofer', 'pupper', 'puppo'], inplace=True)

In [ ]:
# Test 1

tweets_clean.type.value_counts()

In [ ]:
# Test 2

tweets_clean.info()

4.data is splitted in mutliple tables

#### Define
- merge `predictions`, `reactions` tables into `tweets` using the tweet_id

In [ ]:
# Code

tweets_clean = tweets_clean.merge(predictions_clean.loc[:, ['tweet_id','breed', 'conf']], on =['tweet_id'], how ='left')
tweets_clean = tweets_clean.merge(reactions_clean, on =['tweet_id'], how ='left')

In [ ]:
# Test 

tweets_clean.info()

### _Quality_
8.`tweets`, `reactions` some records are retweets

#### Define
- fetch original tweet id for retweets and filter by WeRateDog account
- update like and retweet count based on the tweet_id
- update tweet_id value
- drop reactions for the rest of the retweets

In [ ]:
# Code 

# Removed because none of the retweets ids were actually found

# original_tweet_ids = tweets_clean[~tweets_clean.retweeted_status_id.isna()].copy()
# original_tweet_ids.retweeted_status_id.iloc[0]
#for tweet_id in log_progress(original_tweet_ids.retweeted_status_id):
#     try:
#         while True:
#             try:
#                 result = api.get_status(tweet_id)
#                 break
#             except tp.RateLimitError:
#                 time.sleep(60)
#     except tp.TweepError as e:
#         print(e)
#         continue
#     # override
#     record = original_tweet_ids[original_tweet_ids.retweeted_status_id == tweet_id]
#     record.tweet_id = tweet_id
#     record.retweets = result._json['retweet_count'] 
#     record.likes = result._json['favorite_count']

index = tweets_clean[~tweets_clean.retweeted_status_id.isna()].index
tweets_clean.loc[index, 'retweets'] = np.nan
tweets_clean.loc[index, 'likes'] = np.nan

In [ ]:
# Test 

assert len(tweets_clean[~tweets_clean.retweeted_status_id.isna()][~tweets_clean.retweets.isna()]) == 0

9. breed column contain multiple formats

#### Define
- replace `_` with space
- capitalize the first letter in all words

In [ ]:
# Code 

tweets_clean.breed = tweets_clean.breed.str.replace('_', ' ').str.title()

In [ ]:
# Test

tweets_clean.breed.value_counts()

In [ ]:
#Store cleaned data
tweets_clean.to_csv('data_files/twitter_archive_master.csv')

<hr style="border:2px solid black"> </hr>

# Insights

**which dog breeds tend to get better ratings?**

In [ ]:
tweets_clean = pd.read_csv('data_files/twitter_archive_master.csv')
tweets_clean.groupby('breed').mean().rating.plot(kind ='pie', labels=None)

**which dog breeds are more popular?**

In [ ]:
tweets_clean.groupby('breed').mean().likes.plot(kind ='pie', labels=None)

In [ ]:
tweets_clean.groupby('breed').mean().retweets.plot(kind ='pie', labels=None)

**Taking a closer look**

In [ ]:
tweets_clean.groupby('breed').mean().retweets.nlargest(5).plot(kind ='bar')

In [ ]:
tweets_clean.groupby('breed').mean().likes.nlargest(5).plot(kind ='bar')

In [ ]:
tweets_clean.groupby('breed').mean().rating.nlargest(5).plot(kind ='bar')

**which dog types get more ratings?**

In [ ]:
tweets_clean.groupby('type').mean().rating.plot(kind ='bar')

**which dog types are more popular?**

In [ ]:
tweets_clean.groupby('type').mean().loc[:, ['likes', 'retweets']].plot(kind ='bar')

**how popular are the account tweets based on the given data sample?**

In [ ]:
tweets_clean.sort_values('timestamp').groupby('month_year', sort =False).mean().loc[:, ['likes', 'retweets']].plot(kind='line')

**Does the tweet length affect the tweet popularity?**

In [ ]:
# remove outliers in a simple way
tweets_clean['tweet_len'] = tweets_clean.text.str.len()
tweets_clean[tweets_clean.retweets < 20000].plot.scatter('tweet_len', 'retweets')

In [ ]:
tweets_clean[tweets_clean.likes < 20000].plot.scatter('tweet_len', 'likes')